# Riddler Classic - [November 15, 2019](https://fivethirtyeight.com/features/how-low-can-you-roll/)

## Problem
_You are given a fair, unweighted 10-sided die with sides labeled 0 to 9 and a sheet of paper to record your score. (If the very notion of a fair 10-sided die bothers you, and you need to know what sort of three-dimensional solid it is, then forget it — you have a random number generator that gives you an integer value from 0 to 9 with equal probability. Your loss — the die was a collector’s item.)_

_To start the game, you roll the die. Your current “score” is the number shown, divided by 10. For example, if you were to roll a 7, then your score would be 0.7. Then, you keep rolling the die over and over again. Each time you roll, if the digit shown by the die is less than or equal to the last digit of your score, then that roll becomes the new last digit of your score. Otherwise you just go ahead and roll again. The game ends when you roll a zero._

_For example, suppose you roll the following: 6, 2, 5, 1, 8, 1, 0. After your first roll, your score would be 0.6, After the second, it’s 0.62. You ignore the third roll, since 5 is greater than the current last digit, 2. After the fourth roll, your score is 0.621. You ignore the fifth roll, since 8 is greater than the current last digit, 1. After the sixth roll, your score is 0.6211. And after the seventh roll, the game is over — 0.6211 is your final score._

_What will be your average final score in this game?_

## Analytic Solution
We'll approach this by summing the expected value of each subsequent digit that get's added to our score.

- Digit 1: The first digit can take on values from 0-9 on a uniform distribution so the expected value associates with this digit is $\frac{9}{2*10}=\frac{9}{20}$
- Digit 2: The second digit depends on the value of first digit.  If the first digit was 9, the second digit can take on any value which means the expected value of the second digit is $\frac{9}{2*10^2}$.  Similarly, if the first digit was 8, the expected value of the second digit is $\frac{8}{2*10^2}$ and so on.  Overall the second digit has an expected value of $$\frac{9}{2*10^2}+\frac{8}{2*10^2}+\cdots+\frac{1}{2*10^2}=\frac{9}{2^2*10^2}=\frac{9}{20^2}$$
- Digit i: We can now see the pattern.  The ith digit has an expected value of $$\frac{9}{20^i}$$

Now summing all the digits we see that we have a geometric sequence. 
$$\frac{9}{20^1}+\frac{9}{20^2}+\cdots=\frac{9}{20}*\frac{1}{1-\frac{1}{20}}=9/19=0.47368$$

So on average we will obtain a score of $\frac{9}{19}=0.47368$

## Computational Check
Now lets check this solution with a simple monte carlo simulation. The code below simulates the game and yields the average of $10^6$ simulations.  

In [1]:
import random
import multiprocessing as mp

In [2]:
def generate_number():
    '''
    Simulates one game
    
    Return (float): the score of the game
    '''
    
    # First digit
    prev = random.randint(0,9)
    number = prev/10
    i = 2
    # Generate the second, third, ... digit
    while True:
        d = random.randint(0,9)
        if d == 0: break
        elif d > prev: continue
        else:
            prev = d
            number += d/(10**i)
            i += 1
    return number

We'll use a little parallel processing to speed up our simulation

In [3]:
with mp.Pool(mp.cpu_count()) as pool:
    future_results = [pool.apply_async(generate_number) for i in range(10**6)]
    results = [f.get() for f in future_results]
    print(sum(results)/len(results))

0.4736745090713184


The simulation returns a very similar value giving us confidence that our analytic solution is correct.  